In [1]:
import numpy as np
import pandas as pd

In [2]:
#データの読み込み
train = pd.read_csv('/Users/kanazawakazuki/desktop/SIGNATE/bank_target/train.csv')
train_x = train.drop(['y'], axis=1)
train_y = train['y']
test_x = pd.read_csv('/Users/kanazawakazuki/desktop/SIGNATE/bank_target/test.csv')
sub = pd.read_csv('/Users/kanazawakazuki/desktop/SIGNATE/bank_target/submit_sample.csv', header = None)

In [3]:
# 説明用に学習データとテストデータの元の状態を保存しておく
train_x_saved = train_x.copy()
test_x_saved = test_x.copy()

In [4]:
# 学習データとテストデータを返す関数
def load_data():
    train_x, test_x = train_x_saved.copy(), test_x_saved.copy()
    return train_x, test_x

In [5]:
#必要ではない特徴量を削除する
train_x = train_x.drop(columns=['id', 'job', 'contact', 'pdays', 'previous', 'poutcome', 'default'])
train_x

test_x = test_x.drop(columns=['id', 'job', 'contact', 'pdays', 'previous', 'poutcome', 'default'])
test_x


,age,marital,education,balance,housing,loan,day,month,duration,campaign
0,35,single,secondary,89043,no,no,7,feb,101,2
1,37,married,secondary,64372,yes,no,7,jul,158,3
2,31,single,secondary,31606,yes,no,15,may,152,2
3,31,married,secondary,94826,yes,no,27,may,345,2
4,32,married,secondary,100401,no,no,7,jan,126,1
...,...,...,...,...,...,...,...,...,...,...
18045,49,married,tertiary,98357,yes,no,6,jul,101,2
18046,34,married,secondary,29621,yes,no,12,may,345,1
18047,34,single,secondary,94260,yes,no,16,may,121,2
18048,31,single,secondary,65483,yes,no,15,may,345,2


In [6]:
train_x 

,age,marital,education,balance,housing,loan,day,month,duration,campaign
0,31,married,secondary,12294,yes,no,21,nov,101,3
1,29,single,tertiary,43027,no,no,22,aug,158,2
2,35,married,tertiary,12252,yes,no,11,nov,351,1
3,31,married,secondary,99121,yes,yes,16,may,658,2
4,48,married,primary,42005,yes,no,3,apr,177,1
...,...,...,...,...,...,...,...,...,...,...
27095,37,married,secondary,26661,yes,no,27,may,345,4
27096,35,married,secondary,42150,yes,no,27,may,121,1
27097,35,married,unknown,34531,no,no,28,jun,177,2
27098,30,single,secondary,99621,yes,no,27,may,121,1


In [7]:
test_x 

,age,marital,education,balance,housing,loan,day,month,duration,campaign
0,35,single,secondary,89043,no,no,7,feb,101,2
1,37,married,secondary,64372,yes,no,7,jul,158,3
2,31,single,secondary,31606,yes,no,15,may,152,2
3,31,married,secondary,94826,yes,no,27,may,345,2
4,32,married,secondary,100401,no,no,7,jan,126,1
...,...,...,...,...,...,...,...,...,...,...
18045,49,married,tertiary,98357,yes,no,6,jul,101,2
18046,34,married,secondary,29621,yes,no,12,may,345,1
18047,34,single,secondary,94260,yes,no,16,may,121,2
18048,31,single,secondary,65483,yes,no,15,may,345,2


In [8]:
# 説明用に学習データとテストデータの元の状態を保存しておく
train_x_saved = train_x.copy()
test_x_saved = test_x.copy()


# 学習データとテストデータを返す関数
def load_data():
    train_x, test_x = train_x_saved.copy(), test_x_saved.copy()
    return train_x, test_x


# 変換するカテゴリ変数をリストに格納
cat_cols = ['marital', 'education', 'loan', 'housing']

# -----------------------------------
# one-hot encoding
# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------

# 学習データとテストデータを結合してget_dummiesによるone-hot encodingを行う
all_x = pd.concat([train_x, test_x])
all_x = pd.get_dummies(all_x, columns=cat_cols)

# 学習データとテストデータに再分割
train_x = all_x.iloc[:train_x.shape[0], :].reset_index(drop=True)
test_x = all_x.iloc[train_x.shape[0]:, :].reset_index(drop=True)

# -----------------------------------
# データの読み込み
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoderでのencoding
ohe = OneHotEncoder(sparse=False, categories='auto')
ohe.fit(train_x[cat_cols])

# ダミー変数の列名の作成
columns = []
for i, c in enumerate(cat_cols):
    columns += [f'{c}_{v}' for v in ohe.categories_[i]]

# 生成されたダミー変数をデータフレームに変換
dummy_vals_train = pd.DataFrame(ohe.transform(train_x[cat_cols]), columns=columns)
dummy_vals_test = pd.DataFrame(ohe.transform(test_x[cat_cols]), columns=columns)

# 残りの変数と結合
train_x = pd.concat([train_x.drop(cat_cols, axis=1), dummy_vals_train], axis=1)
test_x = pd.concat([test_x.drop(cat_cols, axis=1), dummy_vals_test], axis=1)


In [7]:
#size_mappingでmonthをそれぞれ1~12に変換する
size_mapping = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 
                'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
train_x['month'] = train_x['month'].map(size_mapping)
test_x['month'] = test_x['month'].map(size_mapping)

train_x['month_day'] = train_x['month']*100 + train_x['day']
test_x['month_day'] = test_x['month']*100 + test_x['day']

In [8]:
#必要ではない特徴量を削除する
train_x = train_x.drop(columns=['day', 'month'])
test_x = test_x.drop(columns=['day', 'month'])

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
train_x

,age,balance,duration,campaign,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,loan_no,loan_yes,housing_no,housing_yes,month_day
0,31,12294,101,3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1121
1,29,43027,158,2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,822
2,35,12252,351,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1111
3,31,99121,658,2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,516
4,48,42005,177,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27095,37,26661,345,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,527
27096,35,42150,121,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,527
27097,35,34531,177,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,628
27098,30,99621,121,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,527


In [11]:
test_x

,age,balance,duration,campaign,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,loan_no,loan_yes,housing_no,housing_yes,month_day
0,35,89043,101,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,207
1,37,64372,158,3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,707
2,31,31606,152,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,515
3,31,94826,345,2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,527
4,32,100401,126,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,98357,101,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,706
18046,34,29621,345,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,512
18047,34,94260,121,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,516
18048,31,65483,345,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,515


In [12]:
scaler = StandardScaler()
num_cols = ['balance', 'duration', 'month_day']
scaler.fit(pd.concat([train_x[num_cols], test_x[num_cols]]))

train_x[num_cols] = scaler.transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

In [13]:
train_x = train_x.astype(float)

In [14]:
train_x

,age,balance,duration,campaign,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,loan_no,loan_yes,housing_no,housing_yes,month_day
0,31.0,-1.114015,-0.621453,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.371973
1,29.0,-0.140798,-0.345405,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.965551
2,35.0,-1.115345,0.589283,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2.324935
3,31.0,1.635521,2.076067,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.473797
4,48.0,-0.173162,-0.253389,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.005321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27095,37.0,-0.659058,0.560226,4.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.422056
27096,35.0,-0.168570,-0.524594,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.422056
27097,35.0,-0.409840,-0.253389,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.053023
27098,30.0,1.651355,-0.524594,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.422056


In [15]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [16]:
import xgboost as xgb
from sklearn.metrics import log_loss

In [17]:
dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)

In [18]:
# ハイパーパラメータの設定
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50

In [19]:
# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
# watchlistには学習データおよびバリデーションデータをセットする
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist)

[0]	train-error:0.075966	eval-error:0.075424
[1]	train-error:0.07567	eval-error:0.073801
[2]	train-error:0.07572	eval-error:0.074391
[3]	train-error:0.075769	eval-error:0.073948
[4]	train-error:0.076359	eval-error:0.074096
[5]	train-error:0.076113	eval-error:0.073801
[6]	train-error:0.07567	eval-error:0.074391
[7]	train-error:0.075375	eval-error:0.073653
[8]	train-error:0.075031	eval-error:0.074391
[9]	train-error:0.074932	eval-error:0.074539
[10]	train-error:0.074391	eval-error:0.074982
[11]	train-error:0.074096	eval-error:0.074391
[12]	train-error:0.073555	eval-error:0.073801
[13]	train-error:0.073604	eval-error:0.073801
[14]	train-error:0.073309	eval-error:0.073948
[15]	train-error:0.073112	eval-error:0.074391
[16]	train-error:0.072768	eval-error:0.074244
[17]	train-error:0.072079	eval-error:0.074244
[18]	train-error:0.07139	eval-error:0.074391
[19]	train-error:0.071095	eval-error:0.073506
[20]	train-error:0.070701	eval-error:0.074096
[21]	train-error:0.070258	eval-error:0.073948
[2

In [20]:
# バリデーションデータでのスコアの確認
va_pred = model.predict(dvalid)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

logloss: 0.2204


In [21]:
# 予測（二値の予測値ではなく、1である確率を出力するようにしている）
pred = model.predict(dtest)
pred

array([0.0917813 , 0.07198502, 0.03123385, ..., 0.07264201, 0.00226069,
       0.13463333], dtype=float32)

In [22]:
sub[1] = pred

sub.to_csv('submission_GBDT.csv',header=False, index=False)